## Lid Driven and free slip boundary conditions in annulus geometry

Very raw model of a annulus geometry in Underworld.

How? A 2D rectangular finite element mesh is build with periodic boundary conditions on the short edges. The rectangel is then deformed into a annulus. The velocity DOFs on the boundary (inner and outer) surfaces to be prescribed by DirichletBCs in normal and tangential components ; the rest of the velocity DOFs remains in standard cartesian coordinates.

This implementation uses elemental rotation matrices in the stiffess matrix and force vector construction to orient the DOF to be aligned, ie normal and tangential, to the boundary of the domain. Currently these rotations are only applied to the boundary DOF. 
Consequently the velocity solution immediately after the stokes solve is a mix of boundary condition aligned to normal & tangent direction, while all other (internal) DOF are prescribed in standard cartesian components. A re-rotation must take place.

In [ ]:
import underworld as uw
from underworld import function as fn
import glucifer
import math
import numpy

In [ ]:
# Set simulation box size.
boxHeight = 1.0
boxLength = 2.0
# Set the resolution.
res = 2
# Set min/max temperatures.
tempMin = 0.0
tempMax = 1.0

# build annulus mesh - handles deforming a recangular mesh and applying periodic dofs
mesh                = uw.mesh._FeMesh_Annulus(elementRes=(10,60), radialLengths=(4,6))

velocityField       = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=2 )
tField              = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=1 )
pressureField       = uw.mesh.MeshVariable( mesh=mesh.subMesh, nodeDofCount=1 )
vmag = fn.math.sqrt(fn.math.dot( velocityField, velocityField ))

In [ ]:
basis=(mesh.rot_vec_normal, mesh.rot_vec_tangent)
len(basis)==velocityField.nodeDofCount

In [ ]:
velocityField.nodeDofCount

In [ ]:
# TODO: reuse only the vertex sets corresponding to the boundaries.
lower = mesh.specialSets["MinI_VertexSet"]
upper = mesh.specialSets["MaxI_VertexSet"]

# (vx,vy) -> (vn,vt) (normal, tangential)
velocityField.data[ upper.data ] = [0.0,10.0]
# velocityField.data[ lower.data ] = [0.0,-6.0]
velBC = uw.conditions.RotatedDirichletCondition( 
                                    variable        = velocityField,
                                    indexSetsPerDof = (lower+upper, upper),
                                    basis_vectors = (mesh.rot_vec_normal, mesh.rot_vec_tangent))


In [ ]:
fig = glucifer.Figure()
fig.append( glucifer.objects.Mesh( mesh ))
# fig.append(glucifer.objects.VectorArrows(mesh, velocityField ))
fig.append(glucifer.objects.Surface(mesh, vmag, onMesh=True  ))
fig.show()

In [ ]:
stokesSLE = uw.systems.Stokes( velocityField = velocityField, 
                            pressureField = pressureField,
                            conditions    = velBC,
                            fn_viscosity  = viscosity, 
                            fn_bodyforce  = buoyancyFn,
                            _removeBCs    = False)      # _removeBC is required
solver = uw.systems.Solver( stokesSLE )

In [ ]:
# stokesSLE.redefineVelocityDirichletBC(fn_e1=mesh.rot_vec_normal, fn_e2=mesh.rot_vec_tangent)

In [ ]:
solver.solve() # results in velocity solution being mixed

In [ ]:
# re-rotate and unmix
uw.libUnderworld.Underworld.AXequalsX( stokesSLE._rot._cself, stokesSLE._velocitySol._cself, False)

In [ ]:
fig = glucifer.Figure()
# fig.append( glucifer.objects.Mesh( mesh ))
fig.append(glucifer.objects.VectorArrows(mesh, velocityField/5. ))
# fig.append(glucifer.objects.Surface(mesh, velocityField[1], onMesh=True  ))
fig.show()

In [ ]:
# create checkpoint function
def checkpoint( mesh, fieldDict, swarm, swarmDict, index,
                meshName='mesh', swarmName='swarm', 
                prefix='./', enable_xdmf=True):
    import os
    # Check the prefix is valid
    if prefix is not None:
        if not prefix.endswith('/'): prefix += '/' # add a backslash
        if not os.path.exists(prefix) and uw.rank()==0:
            print "Creating directory: ",prefix 
            os.makedirs(prefix)
        uw.barrier() 
            
    if not isinstance(index, int):
        raise TypeError("'index' is not of type int")        
    ii = str(index)
    
    if mesh is not None:
        
        # Error check the mesh and fields
        if not isinstance(mesh, uw.mesh.FeMesh):
            raise TypeError("'mesh' is not of type uw.mesh.FeMesh")
        if not isinstance(fieldDict, dict):
            raise TypeError("'fieldDict' is not of type dict")
        for key, value in fieldDict.iteritems():
            if not isinstance( value, uw.mesh.MeshVariable ):
                raise TypeError("'fieldDict' must contain uw.mesh.MeshVariable elements")


        # see if we have already saved the mesh. It only needs to be saved once
        if not hasattr( checkpoint, 'mH' ):
            checkpoint.mH = mesh.save(prefix+meshName+".h5")
        mh = checkpoint.mH

        for key,value in fieldDict.iteritems():
            filename = prefix+key+'-'+ii
            handle = value.save(filename+'.h5')
            if enable_xdmf: value.xdmf(filename, handle, key, mh, meshName)
        
    # is there a swarm
    if swarm is not None:
        
        # Error check the swarms
        if not isinstance(swarm, uw.swarm.Swarm):
            raise TypeError("'swarm' is not of type uw.swarm.Swarm")
        if not isinstance(swarmDict, dict):
            raise TypeError("'swarmDict' is not of type dict")
        for key, value in swarmDict.iteritems():
            if not isinstance( value, uw.swarm.SwarmVariable ):
                raise TypeError("'fieldDict' must contain uw.swarm.SwarmVariable elements")
    
        sH = swarm.save(prefix+swarmName+"-"+ii+".h5")
        for key,value in swarmDict.iteritems():
            filename = prefix+key+'-'+ii
            handle = value.save(filename+'.h5')
            if enable_xdmf: value.xdmf(filename, handle, key, sH, swarmName)


In [ ]:
# xdmf output
fieldDict = {'velocity':velocityField}
checkpoint(mesh, fieldDict, None, None, index=0, prefix='output')